## Electro-Statics with One Dimensional elements

solve

\begin{align}
(\lambda\nabla u, \nabla b)_\Omega &= 0 \quad&&\text{in } \Omega\\
u &= 1 &&\text{on }\Gamma_{\text{top}}\\
u &= 0 &&\text{on }\Gamma_{\text{bottom}}\cup\Gamma_{\text{source}}\\
-\lambda\nabla u\cdot\mathbf{n} &= 0 && \text{on }\Gamma_{\text{left}}\cup\Gamma_{\text{right}}
\end{align}

In [1]:
from netgen . occ import *
from ngsolve import *
from ngsolve.webgui import Draw

In [6]:
W = 3
w1 = 1
H = 2
h1 = 1

lam = 1

orderFE = 2

In [3]:
#        ___________________________________
#       |                                   |        ^
#       |                                   |        |
#       |                                   |        |
#       |       |                   |       |   ^    |
#       |       |                   |       |   |    |
#       |       |                   |       |   |    |
#       |       |                   |       |   h1   H
#       |       |                   |       |   |    |
#       |       |                   |       |   |    |
#       |       |                   |       |   v    |
#       |                                   |        |
#       |                                   |        |
#       0___________________________________|        v
#
#                <--------w1-------->
#        <----------------W----------------->
#
#
#



wp = WorkPlane ( Axes ((0 ,0 ,0) , n =Z , h = X ) )
outer = wp.Rectangle(W , H).Face()

outer = WorkPlane().Rectangle(W, H).Face()
outer.edges.Min(Y).name="bottom"
outer.edges.Max(Y).name="top"
outer.edges.Min(X).name="left"
outer.edges.Max(X).name="right"

left_source = Edge(Segment(Pnt((W-w1)/2, (H-h1)/2, 0), Pnt((W-w1)/2, H-(H-h1)/2, 0)))
left_source.edges.name="source_left"
left_source.edges.maxh=0.05


right_source = Edge(Segment(Pnt(W-(W-w1)/2, (H-h1)/2, 0), Pnt(W-(W-w1)/2, H-(H-h1)/2, 0)))
right_source.edges.name="source_right"
right_source.edges.maxh=0.05

geo = Glue([outer, left_source, right_source])

#Draw(geo, True)

mesh = Mesh ( OCCGeometry ( geo , dim=2).GenerateMesh (maxh = 0.2) )
print(mesh.GetBoundaries())

('bottom', 'left', 'right', 'top', 'source_left', 'source_right')


In [10]:

fes = H1(mesh, order=orderFE, dirichlet="source.*|top|bottom")
u,v = fes.TnT()
sol = GridFunction(fes)
sol.Set(mesh.BoundaryCF({"source.*|bottom":0, "top":1}, default=0), BND)

a = BilinearForm(fes, symmetric=True)
a += lam *grad(u)*grad(v)*dx

prec = Preconditioner(a, type="direct")

solvers.BVP(bf=a, lf=LinearForm(fes), gf=sol, pre=prec)
Draw(sol, settings={"Objects":{"Wireframe":False}})




CG iteration 1, residual = 3.437971537156364     
CG iteration 2, residual = 5.2196953209288885e-15     


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Wireframe': Fal…

BaseWebGuiScene